# Analyse d'e-réputation Twitter — Solution

Ce projet a pour objectif de récapituler (quasi) toutes les compétences vues dans cette formation pour analyser une e-réputation Twitter, en 3 temps :
* Recherche des tweets avec l'API Twitter
* Nettoyage (pre-processing) et analyse des tweets
* Envoi dans un spreadsheet avec l'API Google Sheets.
___

Vous êtes data analyst chez Olist, et vous aimeriez savoir comment vos concurrents sont perçus par le public. Vous codez donc avec Python un programme qui utilise l'API Twitter pour **rechercher les tweets qui évoquent vos concurrents** (par exemple Amazon). Le programme ensuite les **nettoie**, les **analyse** (mots les plus courants, émotions les plus présentes, répartition géographique, sous forme de graphes et de statistiques) et enfin envoie les résultats de ces analyses dans un **spreadsheet**. 

Ce genre d'analyse peut ensuite être **répété**, par exemple chaque semaine, afin de surveiller l'état du marché et **l'évolution de la perception** que les clients ont de vos concurrents (ou de votre entreprise elle-même !). Le projet peut s'adapter à de nombreux usages !

Dans ce notebook, vous serez guidé dans **toutes les étapes** de la création de cet outil. Nous utiliserons les API de Twitter et de Google Sheets, vous aurez donc besoin de **configurer un compte Google Cloud et un compte Twitter Developer** en suivant les instructions disponibles sur la plateforme Databird (**2 fichiers PDF**). Allons-y !!

In [1]:
# exécuter cette cellule si tous les packages ne sont pas installés
!pip install tweepy
!pip install gspread 
!pip install oauth2client
!pip install nltk
!pip install emoji
!pip install wordcloud
!pip install geopy
!pip install folium

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.


  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Uninstalling requests-2.26.0:
      Successfully uninstalled requests-2.26.0


In [ ]:
# packages généralistes
import pandas as pd
import numpy as np
import os
import itertools
from tqdm import tqdm
import datetime

# packages d'analyse de texte
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
import re
from collections import Counter
import emoji

# packages graphiques
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
import folium
from folium.plugins import MarkerCluster, HeatMap
from geopy.geocoders import Nominatim

# API wrappers Twitter et Google Sheets
import tweepy
import gspread

## API Twitter

### Initialisation

In [ ]:
# Remplacez par vos propres clés d'API Twitter
API_KEY = 'diVvpJ68bD2T9Z7oSWcqRFSeh'
API_SECRET_KEY = 'A8WSmaeMahMkUDTI1oG8Q0OSaBCOswxCMCCK4JfkmyzsVBcE0R'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAJXNOgEAAAAAtUAkIFZgT38EsEGEMA%2BWXWZU3n4%3Daa9H6Jd9k7ddlaC92144L55mOqP9Nl4JBHhYLZyufTC50RNbxE'

In [ ]:
# Authentification et création du client d'API
auth = tweepy.AppAuthHandler(API_KEY, API_SECRET_KEY)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

### Recherche

1. **Recherchez les tweets à propos d'Amazon** (ou de **Tesla** — souvent plus amusants, d'expérience). Vous pouvez ajouter d'autres termes de recherche dans votre requête et ajuster les paramètres comme vous le souhaitez. Pour l’instant, limitez votre recherche à **100 tweets**.

_Conseils :_

* Utilisez la fonction `tweepy.Cursor` vue dans le live-coding.
* Lorsque vous recherchez un mot, l’API Twitter le recherche dans toutes données liées à un tweet (texte du tweet, hashtags, nom d’utilisateur, etc.).
* Je vous conseille de filtrer les retweets avec `-filter:retweets` pour obtenir des tweets plus pertinents.
* Le résultat de la fonction `tweepy.Cursor` est un "générateur". Convertissez ce résultat en liste à l’aide de la fonction `list()`.
* Une fois convertis en liste, vous pouvez itérer sur les tweets pour en extraire des informations. Par exemple, pour extraire le texte des tweets, utilisez l’attribut `.full_text` sur les tweets dans votre boucle (voir live-coding).

In [ ]:
query ="tesla -filter:retweets"

In [ ]:
# Recherche des tweets
tweets = tweepy.Cursor(api.search,
                       q=query,
                       lang="en",
                       tweet_mode="extended",
                       #since="2021-06-18", # put here the last week
                       #until="2021-06-25",
                      ).items(50)
tweets = list(tweets)

In [ ]:
# Vérification : extraction du texte des tweets

tweets_text = []
for tweet in tqdm(tweets):
    tweets_text.append(tweet.full_text)
tweets_text

2. Intéressons-nous seulement à **1 tweet** pour l'instant (un tweet complet, dans le résultat de `tweepy.Cursor`, pas seulement le texte d'un tweet). **Extraire les attributs les plus intéressants** de ce tweet à partir de la réponse au format **json**, et stocker ces informations dans une liste.

_Conseils :_

* La réponse json fournie pour chaque tweet dans les résultats contient beaucoup plus d’informations que le simple texte du tweet. Commencez par extraire la réponse json complète d'un seul tweet avec `._json`, pour comprendre comment les informations sont organisées.
* Puis utilisez la syntaxe `.attribut` (`.full_text`, `.user.screen_name`, etc.) pour extraire le texte du tweet, sa date de création, ses hashtags, la localisation de l'utilisateur, le nombre de followers de l'utilisateur, le nombre de statuts publiés par l'utilisateur, le nombre de retweets et le nombre de favoris du tweet.
* Stockez toutes ces informations dans une liste.

In [ ]:
# Réponse json complète pour 1 tweet

test = list(tweets)[10]
json_test = test._json
json_test

On souhaite extraire les attributs : `full_text`, `created_at`, `entities.hashtags`, `user.location`, `user.followers_count`, `user.statuses_count`, `retweet_count`, `favorite_count` :

In [ ]:
# Extraction des informations pertinentes

info = [test.full_text, 
        test.created_at,
        #test.entities['hashtags'], # ne fonctionne pas en l'état ! 
        [h['text'] for h in test.entities['hashtags']], # solution détaillée dessous
        test.user.location, 
        test.user.followers_count, 
        test.user.statuses_count, 
        test.retweet_count, 
        test.favorite_count]
info

Ci-dessous, les quelques tests effectués afin d'écrire la liste compréhension permettant d'extraire les hashtags :

In [ ]:
experience = [{'text': 'dogecoin', 'indices': [39, 48]},
  {'text': 'doge', 'indices': [49, 54]}]
experience

In [ ]:
hashtags = []
for x in experience:
    hashtag = x['text']
    hashtags.append(hashtag)
hashtags

In [ ]:
[h['text'] for h in experience]

3. Répliquez le même processus sur **tous les tweets** issus de la recherche.

_Indices : Vous aurez probablement besoin d'une boucle. Vous pouvez utiliser **`tqdm`** pour afficher une barre de progression. L'objectif est d'obtenir une **liste de listes** (chaque sous-liste correspondant aux informations d'1 tweet)._

In [ ]:
# Extraction des informations pour tous les tweets

tweets_infos = []
for tweet in tqdm(tweets):
    tweet_info = [tweet.full_text, 
                    tweet.created_at, 
                    [h['text'] for h in tweet.entities['hashtags']], 
                    tweet.user.location, 
                    tweet.user.followers_count, 
                    tweet.user.statuses_count, 
                    tweet.retweet_count, 
                    tweet.favorite_count] 
    tweets_infos.append(tweet_info)

In [ ]:
tweets_infos

4. Convertir les résultats en **dataframe** (une ligne par tweet).

In [ ]:
df = pd.DataFrame(tweets_infos)
df.columns = ['full_text', 'created_at', 'hashtags', 'location',
              'followers', 'statuses', 'retweets', 'favorites']
df

### Nettoyage (pre-processing)

On souhaite ici préparer le **texte** des tweets pour l'analyse. Une bonne pratique consiste à **tester toutes les étapes de pre-processing sur 1 seul tweet**, avant de les **appliquer à tous les tweets dans une boucle**. Et c'est ce que nous allons faire !

0. Isolez le texte d'un seul tweet pour pouvoir tester les étapes de pre-processing lors des questions suivantes.

_Conseil : C'est encore mieux si votre tweet contient des @, # et des liens hypertexte, que nous nous efforcerons de supprimer. Si vous ne trouvez pas de tweet qui convient, écrivez simplement un faux tweet de test !_

In [ ]:
test = 'Come spend Christmas 👑 with Amazon! https://t.co/r8oG #Christmas @amazon'

1. Supprimer les liens, identifications (caractère @ et nom d'utilisateur, peu utiles à l'analyse) et tags (caractère # seulement), grâce à la fonction `clean_tweet` ci-dessous.

In [ ]:
def clean_tweet(txt):
    return re.sub("(http\S+)|(@\S+)", "", txt).replace('#', '')

In [ ]:
test = clean_tweet(test)
test

2. Mettre le texte en minuscules.

In [ ]:
test = test.lower()
test

3. Supprimer tous les signes de ponctuation et les remplacer par des espaces.

_Conseils : Vous aurez sans doute besoin d'une "regex" (expression régulière), comme dans la fonction `clean_tweet`. Attention à ne pas supprimer les emojis. Vous pouvez utiliser la regex : `re.sub(r'[^\w\s\U00010000-\U0010ffff]', ' ', test_tweet)`. Cette regex signifie : "remplacer tous les caractères qui ne sont pas (`^`) des lettres (`\w`), des espaces (`\s`) ou des caractères unicode (`\U`, sur tous les codes possibles) par un espace"._

In [ ]:
test = re.sub(r'[^\w\s\U00010000-\U0010ffff]', ' ', test)
test

4. Scinder le texte en mots individuels et supprimer les espaces avant et après chaque mot ("leading and trailing white spaces") si besoin.

In [ ]:
test = test.split()
test  # .strip() à appliquer sur tous les mots si besoin

5. Supprimer les mots de la requête ("collection words").

_Indice : Vous pouvez utiliser une list comprehension incluant une condition._

In [ ]:
collection_words = ['amazon']
test = [ word for word in test if not word in collection_words ]
test

6. Supprimer les "stopwords", c'est-à-dire les mots courants qui ne portent pas de sens ("the", "a", etc.).

In [ ]:
# Télécharger la liste de stopwords du package nltk 
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(list(stop_words)[0:10])

In [ ]:
test = [ word for word in test if not word in stop_words ]
test

7. Appliquer les étapes précédentes à tous les tweets. L'objectif est d'obtenir un long corpus de texte, i.e. une longue liste contenant tous les mots indicviduels issus des tweets nettoyés.

_Conseils :_
* Commencez par créer une liste contentant le texte de tous vos tweets.
* Utilisez ensuite une boucle pour appliquer les étapes de nettoyage à chaque tweet.
* Souvenez-vous de la différence entre `.append` et `.extend`.

In [ ]:
# A partir du résultat de la recherche, conserver seulement le texte des tweets
tweets_text = [t[0] for t in tweets_infos]
tweets_text

In [ ]:
# Construire le corpus de mots en appliquant les étapes testées plus haut
corpus = []
for tweet in tweets_text:
    clean = clean_tweet(tweet).lower()
    clean = re.sub(r'[^\w\s\U00010000-\U0010ffff]', ' ', clean)
    clean = clean.split()
    collection_words = ['tesla']
    clean = [word for word in clean if not word in collection_words]
    clean = [word for word in clean if not word in stop_words]
    corpus.extend(clean)

In [ ]:
corpus

In [ ]:
# Version fonctionnelle

def clean_full(tweet: str):
    clean = clean_tweet(tweet).lower()
    clean = re.sub(r'[^\w\s\U00010000-\U0010ffff]', ' ', clean)
    clean = clean.split()
    collection_words = ['amazon']
    clean = [word for word in clean if not word in collection_words]
    clean = [word for word in clean if not word in stop_words]
    return clean

In [ ]:
corpus = []
for tweet in tweets_text:
    result = clean_full(tweet)
    corpus.extend(result)

8. Excellent, nous avons désormais un corpus de mots prêt à l'analyse ! Pour cette dernière question de la section, créer un nouvelle colonne dans le dataframe de la section précédente. Mettre dans cette colonne, le text des tweets, en leur appliquant uniquement les étapes 1 and 2 du cleaning (supprimer les @, # et les links, et mettre en minuscules). Nommer cette colonne `clean_text`. Nous l'utilserons aussi pour l'analyse.

_Conseil : Vous pouvez utiliser `apply`._

In [ ]:
df['clean_text'] = df['full_text'].apply(lambda x: clean_tweet(x).lower())

In [ ]:
df

### Analyse

Dans cette section, nous mènerons plusieurs analyses fondées sur le corpus de mots et sur le dataframe contenant les tweets complets nettoyés, ainsi que les autres informations issues de notre recherche.

#### Mots les plus fréquents

0. Compter le nombre de mots uniques dans le corpus.

In [ ]:
print(f"There are {len(set(corpus))} words in the combination of all searched tweets.")

1. Compter le nombre d'occurences de chaque mot. Quels sont les 10 mots les plus fréquents ?

_Indices : Souvenez-vous de `Counter()` du package `collections`. Essayez-aussi d'appliquer `.most_common(10)` au résultat de votre compteur._

In [ ]:
top10_words = Counter(corpus).most_common(10)
top10_words

In [ ]:
pd.Series(corpus).value_counts()

2. Afficher la fréquence des n premiers mots du corpus (n doit être modifiable) dans un diagramme en bâtons (bar plot).

_Indice : Vous pouvez créer le bar plot manuellement, ou bien utiliser la fonction du notebook solutions (plus rapide !)._

In [ ]:
def plot_frequencies(words, top=15):
    
    # Create dataframe for plotting
    frequency_df = pd.DataFrame(Counter(words).most_common(top),
                             columns=['words', 'count'])
    
    # Plot
    frequency_df.sort_values(by='count').plot.barh(x='words',
                                                   y='count',
                                                   color="red")
    plt.title("Most Common Words in Tweets", fontweight='semibold')
    plt.show()

In [ ]:
plot_frequencies(corpus, top=10)

3. Utiliser la fonction ci-dessous pour afficher un nuage de mots (wordcloud) des mots les plus communs dans le corpus.

_Note : Vous pouvez inclure un argument `mask` dans la fonction, avec le chemin d'une image qui donnera forme au nuage de mots. Il peut s'agir de (quasi) n'importe quelle image en noir et blanc — en particulier, vous pouvez utiliser le fichier `glasses.jpg` disponible sur la plateforme._

In [ ]:
def wordcloud(corpus, title=None, mask=None, figsize=(10, 10)):
    """
    Draws a word cloud with an optional mask shape, using the wordcloud package.

    Parameters
    ----------
    
    corpus: list
        List of all individual words to be used for the word cloud.
    
    title: string, optional, default: None
        The title of your word cloud.

    mask: string, optional, default: None
        The path to the image to be used as mask. It must be binary (black & white).
        
    figsize: tuple, optional, default: (10,10)
        The size of the chart area.
    """
    
    # preprocess data
    corpus = str(corpus).replace("'", "")
    
    # load mask if provided
    if mask != None:
        mask_ = Image.open(mask)
        fn = lambda x : 255 if x >= 200 else 0
        mask_ = mask_.convert('L').point(fn, mode='1').convert('RGB')
        mask_ = np.array(mask_)
    else: 
        mask_ = None
    
    wordcloud = WordCloud(
        background_color = 'white',
        mask = mask_,
        max_font_size = 45,
        min_font_size = 5,
        contour_width = 0.1,
        contour_color = 'silver',
        repeat = True,
        stopwords=STOPWORDS,
        random_state = 1).generate(str(corpus))
    
    fig = plt.figure(1, figsize=figsize)
    plt.axis('off')
    if title:
        fig.suptitle(title, fontsize=18, y=0.75)
    plt.imshow(wordcloud)

In [ ]:
wordcloud(corpus, title="Most Common Words in Tweets")

In [ ]:
wordcloud(corpus, title="Most Common Words in Tweets", mask='glasses.jpg')

#### Hashtags et emojis

1. Quels sont les hashtags les plus fréquents dans nos tweets ? (top 10)

_Indices : Vous aurez sans doute besoin "d'aplatir" ("flatten") une liste de listes, c'est-à-dire transformer une liste de listes en une seule longue liste. Pour cela, vous pouvez utiliser le package `itertools` : `simple_list = list(itertools.chain(*list_of_lists))`. Souvenez-vous aussi de mettre tous les hashtags en minuscules._

In [ ]:
hashtags = list(df['hashtags'].values)
hashtags

In [ ]:
hashtags = list(itertools.chain(*hashtags))
hashtags

In [ ]:
# mettre en minuscules pour éviter les doublons
hashtags = [ h.lower() for h in hashtags ]

In [ ]:
top10_hashtags = Counter(hashtags).most_common(10)
top10_hashtags

2. Quels sont les emojis les plus fréquents ? (top 10)

_Indice : La première étape pour cette question est de créer un corpus d'emojis en conservant seulement les emojis dans le corpus de mots complet. Vous pouvez pour cela utiliser la fonction `char_is_emoji` ci-dessous._

In [ ]:
def char_is_emoji(character):
    return character in emoji.UNICODE_EMOJI['en']

In [ ]:
emoji_corpus = []
for word in corpus:
    for character in word:
        if char_is_emoji(character):
            emoji_corpus.append(character)

In [ ]:
emoji_corpus

In [ ]:
top10_emojis = Counter(emoji_corpus).most_common(10)
top10_emojis

#### Analyse de sentiment

Parlons maintenant de **NLP, "Natural Language Processing"** (parfois traduit "Traitement Automatique du Langage Naturel", TALN). Le NLP est un champ de recherche du Deep Learning, l'apprentissage profond, qui s'intéresse à l'analyse de texte et à la construction de modèles prédictifs liés au langage. En particulier, un important domaine d'application du NLP est l'analyse de sentiment, i.e. **prédire le sentiment transmis par une phrase ou un paragraphe**. 

Nous allons utiliser l'analyse de sentiment sur nos tweets pour essayer de déterminer les **sentiments les plus communs quand les internautes évoquent notre sujet sur Twitter**.

La construction d'un modèle de NLP est relativement complexe, heureusement le package `nltk` nous fournit plusieurs fonctions de NLP, notamment une **fonction pour faire appel à un modèle d'analyse de sentiment pré-entraîné**. Exécutez la cellule ci-dessous pour initialiser ce modèle, puis suivez les questions !

In [ ]:
# Initialisation du modèle d'analyse de sentiment
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

1. Prédire le sentiment d'un seul tweet avec la commande `sia.polarity_scores(test_tweet)`. Le tweet passé en argument doit être une phrase complète, i.e. une longue chaîne de caractères, et non une liste de mots.

_Note : Comme vous le verrez, le sentiment est mesuré en pourcentage, réparti entre 'négatif', 'positif' et 'neutre'. La commande renvoie aussi un score composite ('compound'), compris entre -1 et 1. Des explications supplémentaires sur le modèle et le calcul des scores sont disponibles sur le GitHub du modèle Vader utilisé par nltk : [link](https://github.com/cjhutto/vaderSentiment#about-the-scoring)._

In [ ]:
test = 'Come spend the Christmas period 👑 with Amazon !'

In [ ]:
sia.polarity_scores(test)

2. Calculer le sentiment moyen de tous les tweets.

_Indice : Utilisez la colonne `clean_text` de votre dataframe. Le score qui nous intéresse ici est le score "compound"._

In [ ]:
clean_tweets = df['clean_text'].values
clean_tweets

In [ ]:
sentiment_scores = [ sia.polarity_scores(t)["compound"] for t in clean_tweets ]
sentiment_scores

In [ ]:
mean_sentiment = np.mean(sentiment_scores)
mean_sentiment

3. Afficher la répartition des sentments sous forme de graphe (histogramme) grâce à la fonction ci-dessous.

_Note : La majorité des tweets seront sans doute classés comme "neutres", car le modèle n'a pas été entraîné sur des données tout à fait similaires aux nôtres, et nos données ne sont pas préparées de manière tout à fait adéquate pour le modèle — les résultats ne sont donc qu'indicatifs, mais cette première approche du NLP me semble intéressante néanmoins !_

In [ ]:
# sentiment analysis plot

def plot_sentiments(tweets):
    
    sia = SentimentIntensityAnalyzer()
    
    scores = []
    for t in clean_tweets:
        scores.append(sia.polarity_scores(t)['compound'])
    
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.hist(scores, bins=3, range=(-1,1), color='red')
    ax.set_title("Most Common Sentiments in Tweets")
    ax.set_xticks([-0.7, 0, 0.7])
    ax.set_xticklabels(['Negative', 'Neutral', 'Positive'])
    plt.show()

In [ ]:
plot_sentiments(clean_tweets)

#### Popularité

Par popularité, on entend "à quel point les discussions à propos de notre sujet sont populaires", en termes de nombre de retweets et de mises en favoris, et en termes de nombre followers et nombre de statuts des utilisateurs qui postent à ce sujet. Dans cette section, nous analyserons donc les colonnes `retweets`, `favorites`, `followers` et `statuses` de notre dataframe.

1. Calculer le nombre moyen de followers et de statuts des utilisateurs qui tweetent à propos de notre sujet (= la popularité de notre communauté). La médiane serait-elle ici plus pertinent que la moyenne ?

In [ ]:
mean_followers = df['followers'].median()
mean_statuses = df['statuses'].median()
print(mean_followers)
print(mean_statuses)

2. Calculer la moyenne du nombre de retweets et de favoris de nos tweets (= la popularité des tweets sur notre sujet).

In [ ]:
mean_retweets = df['retweets'].mean()
mean_favorites = df['favorites'].mean()
print(mean_retweets)
print(mean_favorites)

3. Quel est le tweet le plus populaire de notre requête ?

_Indice : On peut considérer que le tweet le plus populaire est celui qui a le plus grand nombre cumulé de retweets et de favoris._

In [ ]:
df['sum_favorites_retweets'] = df['favorites'] + df['retweets']
top_tweet = df[df['sum_favorites_retweets'] == max(df['sum_favorites_retweets'])]
top_tweet

In [ ]:
top_tweet = top_tweet['full_text'].values[0]
top_tweet

#### Localisations

Nous analyserons dans cette section l'origine des auteurs de nos tweets — lorsque cette localisation est connue (dépend des informations fournies par l'utilisateur dans son profil).

1. Extraire la colonne `locations` de notre dataframe de tweets, et la stocker sous forme de liste.

In [ ]:
locations = df['location'].values
locations

2. Utiliser la fonction `Nominatim()` du package `geopy.geocoders` pour effectuer un "reverse geocoding" ("géocodage inverse") des localisations.


_Note : Le géocodage inverse consiste à obtenir la latitude et la longitude d'un lieu à partir de son nom. Par exemple, la fonction `Nominatim()` peut prendre comme argument "New York, US" et renvoyer la latitude et la longitude du centre de New York. En coulisses, cette fonction requête l'API de géocodage inverse "Nominatim" qui est gérée par OpenStreetMap, une version open-source de Google Maps._

_Conseils :_

* L'objectif de cette question est de créer un **dictionnaire** contenant 2 clés : "latitude" et "longitude", et à l'intérieur de chaque clé une **liste** de toutes les latitudes (ou longitudes) de nos localisations.
* Regardez l'exemple ci-dessous pour voir comment reverse géocoder une seule localisation. Vous pourrez ensuite faire de même sur toutes les localisations dans une boucle, en remplissant au fur et à mesure votre dictionnaire de résultats.

In [ ]:
# Initialisation
geolocator = Nominatim(user_agent="my-app")

# Exemple
location = 'New York, US'
reverse_location = geolocator.geocode(location)
lat = reverse_location.latitude
lon = reverse_location.longitude
print(lat, lon)

In [ ]:
# Boucle sur toutes les localisations pour obtenir leurs coordonnées

coordinates = {'latitude': [], 'longitude': []}

for user_loc in tqdm(locations):
    try:
        location = geolocator.geocode(user_loc)
        coordinates['latitude'].append(location.latitude)
        coordinates['longitude'].append(location.longitude)         
    except:
        continue

coordinates

3. Transformer le dictionnaire de la question précédente en dataframe (de 2 colonnes).

In [ ]:
df_loc = pd.DataFrame(coordinates)
df_loc

4. Utiliser la fonction ci-dessous pour afficher les localisations sur une carte interactive.

_Note : La fonction utilise le package `folium` pour créer la carte. Pas la peine de tout comprendre dans la fonction, il s'agit de détails de syntaxe liés à ce package ; en revanche il est intéressant de lire attentivement la "docstring", c'est-à-dire  la description de la function, afin de comprendre quels paramètres doivent être fournis._

In [ ]:
def map_locations(df, lat, lon, kind='heatmap', save=False, zoom=2):
    """
    Displays a map with 2 layers (OSM and ESRI satellite imagery) and a heatmap or markers
    corresponding to the locations passed as a dataframe of latitudes and longitudes.

    Parameters
    ----------
    
    df: dataframe
        A dataframe containing at least 2 columns, corresponding to the latitude and 
        longitudes of the points you want to plot.
        
    lat: string
        The name of the column containing the latitudes.
    
    lon: string
        The name of the column containing the longitudes.
    
    kind: string, either 'heatmap' (default) or 'markers'
        The kind of visualization you want to plot.
    
    save: boolean, optional, default: False
        Save the map as 'map.html' in the current folder.
        
    zoom: int, between 0 and 20, optional, default: 2
        The zoom of the base map. Lower is less zoomed.
    """
    
    # Create map, with default OSM tile layer
    m = folium.Map(location=[df[lat].mean(), df[lon].mean()], zoom_start=zoom)

    # Create markers
    locations = []
    popups = []
    for idx, row in df.iterrows():
        locations.append([row[lat], row[lon]])
        popups.append(([row[lat], row[lon]]))

    # Add markers
    if kind == 'markers':
        s = folium.FeatureGroup(name='Points')
        s.add_child(MarkerCluster(locations=locations, popups=popups))
        m.add_child(s)

    # Plot heatmap
    elif kind == 'heatmap':
        heatmap = df[[lat, lon]].values
        m.add_child(HeatMap(heatmap, name='Heatmap', radius=12))
    else:
        print('Please specify a valid kind of map ("markers" or "heatmap").')
    
    # Add ability to see lat and lon onclick
    m.add_child(folium.LatLngPopup())

    # Add satellite layer
    tile = folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            overlay = False,
            control = True
           ).add_to(m)

    # Add layer control toggle
    # We can control the display of both tile layers and overlay figures
    folium.LayerControl().add_to(m)
    
    # Save map as HTML
    if save:
        m.save("map.html")
    
    return m

In [ ]:
map_locations(df_loc, 'latitude', 'longitude', kind='markers', save=True)

#### Résutats finaux

Comme résultat final de cette analyse, nous voulons une **liste** contenant les principaux résultats de notre analyse.

Nous voulons que tout soit dans une liste unique, afin de pouvoir la **sauvegarder** puis utiliser l'API Google Sheets (section suivante) pour l'envoyer directement dans un **spreadsheet**, où elle constituera une **nouvelle ligne**.

De cette façon, nous serons en mesure d'exécuter l'analyse à nouveau à intervalles réguliers, et d'ajouter simplement à chaque fois une nouvelle ligne de résultats à notre Google Sheets, afin de suivre l'évolution de notre sujet au fil du temps.

1. Créer une liste finale contenant 13 éléments (dans cet ordre) :

* la date de l'analyse (aujourd'hui), sous forme de string
* le nombre de tweets que nous avons analysés
* les 10 mots les plus fréquents sous forme de liste
* les 10 emojis les plus fréquents sous forme de liste
* les 10 hashtags les plus fréquents sous forme de liste
* le sentiment moyen de nos tweets (score compound)
* le nombre moyen de followers
* le nombre moyen de statuts
* le nombre moyen de retweets
* le nombre moyen de favoris
* le tweet le plus populaire
* la liste de latitudes
* la liste de longitudes

_Indices : Nous avons déjà tout calculé ici, à l'exception de la date d'aujourd'hui ! Vous devez juste rassembler tous les résultats précédents (et éventuellement ajuster leur format). Pour la date d'aujourd'hui, vous pouvez utiliser `import datetime` puis `str(datetime.datetime.now())`._

In [ ]:
import datetime

In [ ]:
final_insights = [str(datetime.datetime.now()),
                  len(df),
                  [t[0] for t in top10_words],
                  [t[0] for t in top10_emojis],
                  [t[0] for t in top10_hashtags],
                  mean_sentiment,
                  mean_followers,
                  mean_statuses,
                  mean_retweets,
                  mean_favorites,
                  top_tweet,
                  coordinates['latitude'],
                  coordinates['longitude'],
                 ]
final_insights

2. Convertir la liste finale en string et la sauvegarder sous forme de fichier texte nommé `analysis.txt`.

_Conseils :_

* Vous devrez utiliser la syntaxe `with open()`. Voici un exemple de cette syntaxe, à adapter à votre cas :

```
with open(filename, 'w') as f:
    f.write(content)
```

* Certains emojis peuvent provoquer des erreurs d'encoding lorsqu'ils sont sauvegardés dans un ficher texte. Si c'est le cas, vous pouvez utiliser la ligne suivante au moment de l'écriture de votre fichier, afin de convertir les emojis en caractères unicode : `f.write(str(content).encode('unicode-escape').decode("utf-8"))`.

In [ ]:
with open('analysis.txt', 'w') as f:
    f.write(str(final_insights))

## API Google Sheets

Dans cette dernière section, vous serez guidés pour la dernière étape de ce projet Twitter : **envoyer les résultats de l'anayse dans un fichier Google Sheets**. L'objectif est de construire la brique finale de notre workflow, afin d'avoir un **code complet prêt à être automatisé ou réutilisé sur d'autres sujets**. 

Nous avons choisi Google Sheets car c'est un format facilement partageable, compris par tous les utilisateurs (même non-tech), et surtout parce que vous pouvez facilement l'utiliser pour **construire un dashboard de suivi** — par exemple avec **Data Studio**, un dashboard qui serait automatiquement mis à jour chaque semaine pour suivre visuellement la perception de votre sujet au fil du temps.

### Initialisation

Nous avons d'abord besoin de connecter ce notebook à votre **compte de service Google Cloud** (utilisez le PDF d'instructions disponible sur la plateforme créer un compte si besoin) grâce au package **`gspread`**.

Pour vous authentifier, copiez-collez ci-dessous le **chemin de la clé** (fichier .json) qui a été téléchargée au moment de la création de votre compte.

In [ ]:
# Votre clé devrait ressembler à ceci :
path = "/Users/Thomas/Downloads/databird-309423-c4ff87ee5aec.json"

In [ ]:
# Authentification avec gspread
gc = gspread.service_account(filename=path)

### Chargement des résultats

1. Lire le fichier `analysis.txt` qui a été sauvegardé à la fin de l'analyse Twitter.

_Indice : Vous pouvez ouvrir le fichier avec la fonction `open()`, puis le lire avec la méthode `.read()`. Ci-dessous, un exemple de syntaxe, à adapter à votre cas :_

```
f = open(filepath, 'r')
f = f.read()
f
```

In [ ]:
analysis_path = '/Users/Thomas/Documents/Data Science X2/DataBird/Batch4-5/3. Python/J9 APIs 1/Cas Twitter/analysis.txt'

In [ ]:
file = open(analysis_path, "r")
file = file.read()
file

2. Quel est le type du fichier que vous venez de lire (type python) ? Convertissez-le en liste.

_Indice : Vous aurez besoin de la fonction `ast.literal_eval()`._

In [ ]:
type(file)

In [ ]:
import ast
row = ast.literal_eval(file)
row

In [ ]:
type(row)

3. La liste contient différents types de données (string, int, dictionnaire, liste). Hors Google Sheets n'acceptera pas de mettre un dictionnaire ou une liste dans une cellule (c'est comme dans Excel : les seuls types de données que vous pouvez avoir dans une cellule sont du texte ou des chiffres, ou tout au plus des dates). Comment résoudriez-vous ce problème ?

In [ ]:
type(row[1])

In [ ]:
# convertir tous les éléments non-numériques de la liste en strings
formatted_row = [str(x) if not (type(x) == int or type(x) == float) else x for x in row]
formatted_row

### Envoi au spreadsheet

Dans cette section, nous ouvrons une feuille de calcul Google Sheets avec `gspread` et nous y ajoutons une nouvelle ligne contenant la liste de résultats. Assurez-vous de bien **créer d'abord cette feuille de calcul** et **d'autoriser l'API Google Sheets**, en suivant les **instructions** disponibles sur la plateforme Databird (fichier PDF).

1. Ouvrez votre spreadsheet avec `gspread` (aidez-vous du notebook de démo si besoin).

In [ ]:
worksheet = gc.open_by_key('1c_Eij3Y-fAX7qaOQKSFOBivtV5Zb3oNprY3qE_PiNZU').sheet1
worksheet

2. Ajoutez les résultats de l'analyse Twitter (c'est-à-dire la liste) en tant que nouvelle ligne de votre feuille de calcul (à nouveau, aidez-vous de la démo pour utiliser la méthode la plus adéquate !).

In [ ]:
worksheet.append_row(formatted_row)

[BONUS] 3. Ecrivez une longue fonction qui reprend toutes les étapes du projet (de la recherche initiale à l'envoi dans Google Sheets) pour pouvoir appliquer le workflow complet sur une nouvelle recherche de tweets (à passer en argument de la fonction).

In [ ]:
import tweepy
import nltk
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from geopy.geocoders import Nominatim
from tqdm import tqdm
import datetime
import pandas as pd
import numpy as np
import re
from collections import Counter
import itertools
import emoji
import gspread
nltk.download('vader_lexicon')
nltk.download('stopwords')

def twitter_analysis(query_params={'q':'tesla',
                                   'lang':'en',
                                   'since':"2021-04-16",
                                   'until':"2021-04-17"}, 
                     full_output=False,
                    ):
    
    print('Searching for tweets...')
    
    # Twitter API keys
    API_KEY = 'diVvpJ68bD2T9Z7oSWcqRFSeh'
    API_SECRET_KEY = 'A8WSmaeMahMkUDTI1oG8Q0OSaBCOswxCMCCK4JfkmyzsVBcE0R'
    BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAJXNOgEAAAAAtUAkIFZgT38EsEGEMA%2BWXWZU3n4%3Daa9H6Jd9k7ddlaC92144L55mOqP9Nl4JBHhYLZyufTC50RNbxE'
    
    # Twitter authentication
    auth = tweepy.AppAuthHandler(API_KEY, API_SECRET_KEY)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
    # Twitter search
    tweets = tweepy.Cursor(api.search,
                           tweet_mode="extended",
                           **query_params,
                          ).items(50)
    tweets = list(tweets)
    
    print('Analyzing tweets...')
    
    # Information extraction
    tweets_infos = []
    for tweet in tqdm(tweets):
        tweet_info = [tweet.full_text, 
                        tweet.created_at, 
                        [h['text'] for h in tweet.entities['hashtags']], 
                        tweet.user.location, 
                        tweet.user.followers_count, 
                        tweet.user.statuses_count, 
                        tweet.retweet_count, 
                        tweet.favorite_count] 
        tweets_infos.append(tweet_info)
    
    # Dataframe creation
    df = pd.DataFrame(tweets_infos)
    df.columns = ['full_text', 'created_at', 'hashtags', 'location',
                  'followers', 'statuses', 'retweets', 'favorites']
    df['clean_text'] = df['full_text'].apply(lambda x: re.sub("(http\S+)|(@\S+)", "", x).replace('#', '').lower())
    
    # Tweet cleaning / corpus creation
    tweets_text = [t[0] for t in tweets_infos]
    corpus = []
    for tweet in tweets_text:
        clean = re.sub("(http\S+)|(@\S+)", "", tweet).replace('#', '').lower()
        clean = re.sub(r'[^\w\s\U00010000-\U0010ffff]', ' ', clean)
        clean = clean.split()
        collection_words = ['amazon']
        clean = [word for word in clean if not word in collection_words]
        stop_words = set(stopwords.words('english'))
        clean = [word for word in clean if not word in stop_words]
        corpus.extend(clean)
    
    # Word frequency analysis
    top10_words = Counter(corpus).most_common(10)

    # Sentiment analysis
    sia = SentimentIntensityAnalyzer()
    clean_tweets = df['clean_text'].values
    sentiment_scores = [sia.polarity_scores(t)["compound"] for t in clean_tweets]
    mean_sentiment = np.mean(sentiment_scores)
    
    # Hashtags analysis
    hashtags = list(df['hashtags'].values)
    hashtags = list(itertools.chain(*hashtags))
    hashtags = [h.lower() for h in hashtags]
    top10_hashtags = Counter(hashtags).most_common(10)
    
    # Emojis analysis
    emoji_corpus = []
    for word in corpus:
        for character in word:
            if character in emoji.UNICODE_EMOJI['en']:
                emoji_corpus.append(character)
    top10_emojis = Counter(emoji_corpus).most_common(10)
    
    # Popularity analysis
    mean_followers = df['followers'].mean()
    mean_statuses = df['statuses'].mean()
    mean_retweets = df['retweets'].mean()
    mean_favorites = df['favorites'].mean()
    df['sum_favorites_retweets'] = df['favorites'] + df['retweets']
    top_tweet = df[df['sum_favorites_retweets'] == max(df['sum_favorites_retweets'])]
    top_tweet = top_tweet['full_text'].values[0]
    df = df.drop(columns=['sum_favorites_retweets'])
    
    # Location coordinates
    locations = df.location.values
    geolocator = Nominatim(user_agent="my-app")
    coordinates = {'latitude': [], 'longitude': []}
    for user_loc in tqdm(locations):
        try:
            location = geolocator.geocode(user_loc)
            if location:
                coordinates['latitude'].append(location.latitude)
                coordinates['longitude'].append(location.longitude)         
        except:
            pass
    df_loc = pd.DataFrame(coordinates)
    
    # Final output
    final_insights = [str(datetime.datetime.now()),
                  len(df),
                  [t[0] for t in top10_words],
                  [t[0] for t in top10_emojis],
                  [t[0] for t in top10_hashtags],
                  mean_sentiment,
                  mean_followers,
                  mean_statuses,
                  mean_retweets,
                  mean_favorites,
                  top_tweet,
                  coordinates['latitude'],
                  coordinates['longitude'],
                 ]
    
    print('Sending to Google Sheets...')
    
    # Google Sheets authentication
    path = "/Users/Thomas/Downloads/databird-309423-c4ff87ee5aec.json"
    gc = gspread.service_account(filename=path)
        
    # Prepare final ouput for sending
    formatted_row = [str(x) if not (type(x) == int or type(x) == float) else x for x in final_insights]
    
    # Send to Google sheets
    worksheet = gc.open_by_key('1c_Eij3Y-fAX7qaOQKSFOBivtV5Zb3oNprY3qE_PiNZU').sheet1
    worksheet.append_row(formatted_row)
    
    print('Done!')

    if full_output:
        return final_insights, df
    else:
        return final_insights

In [ ]:
final_insights, full_output = twitter_analysis(query_params={'q':'tesla',
                                                             'lang':'en',
                                                             'since':"2021-04-16",
                                                             'until':"2021-04-17"},
                                               full_output=True)

In [ ]:
final_insights

In [ ]:
full_output

Félicitations, vous disposez maintenant d'un très bon outil d'analyse ! Vous pourriez placer le script complet dans un **fichier python .py** et l'exécuter périodiquement (par exemple chaque semaine), grâce à un **planificateur** de script comme **Airflow** (le plus connu). Vous auriez bientôt plusieurs lignes dans votre Google Sheets, que vous pourriez directement utiliser pour **construire et partager des dashboards d'analyse** (grâce à Data Studio) avec toute votre entreprise !